# 下載檔案 第一次做就好

### Download GitHub files

In [1]:
!git clone https://github.com/openaifab/suicide_multiclass.git

Cloning into 'suicide_multiclass'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 34 (delta 0), reused 0 (delta 0), pack-reused 31
Unpacking objects: 100% (34/34), done.



### 下載這個google Drive資料夾裡面的兩個model檔案，都放在suicide_multiclass資料夾裡:


https://drive.google.com/drive/folders/11bmONk9eofHhV5DuwLGjk5mBZCaXRzHt?usp=sharing

In [2]:
# check TensorFlow version is 2.X
import tensorflow as tf
print(tf.__version__)

2.3.0


# Load the model

### Import libraries

In [3]:
#以後都在這個目錄下工作 "suicide_multiclass"
%cd suicide_multiclass
%pwd

/content/suicide_multiclass


'/content/suicide_multiclass'

In [4]:
import numpy as np
import pandas as pd
import pickle
from gensim.models import word2vec
import random
import tensorflow as tf
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

from OpenFabLibrary import JeibaCutWords
from OpenFabLibrary import process_df

### Set parameters

In [9]:
trunc_type='post'
padding_type='pre' #'pre'
oov_tok = "<OOV>"
max_length = 500 # 選取多少詞來當作輸入 Ex:選前面40個詞當作輸入
WORD2VEC_DIMENTION = 512 

### Load the tokenizer

In [8]:
with open("tokenizer", 'rb') as file:
    tokenizer = pickle.load(file)

In [10]:
word_index = tokenizer.word_index
vocab_size = len(word_index)
index_word = tokenizer.index_word

print(f'vocab size is: {vocab_size}')
print(word_index)

vocab size is: 21070
{'<OOV>': 1, '自殺': 2, '想死': 3, '憂鬱': 4, '死': 5, '醫生': 6, '佛': 7, '痛苦': 8, '憂鬱症': 9, '朋友': 10, '情緒': 11, '念佛': 12, '講': 13, '一次': 14, '問': 15, '文': 16, '問題': 17, '人生': 18, '應該': 19, '無法': 20, '諮商': 21, '哭': 22, '努力': 23, '彌陀': 24, '水桶': 25, '睡': 26, '看到': 27, '生活': 28, '藥': 29, '板': 30, '愛': 31, '文章': 32, '家人': 33, '心理': 34, '阿彌陀': 35, '不好': 36, '狀況': 37, '阿彌陀佛': 38, '不到': 39, '生': 40, '下去': 41, '念': 42, '精神': 43, '母親': 44, '第一': 45, '辦法': 46, '發現': 47, '前': 48, '吃藥': 49, '媽': 50, '需要': 51, '發文': 52, '最後': 53, '好好': 54, '一起': 55, '一定': 56, '月': 57, '最': 58, '根本': 59, '直接': 60, '自殘': 61, '心': 62, '治療': 63, '最近': 64, '正常': 65, '回家': 66, '告訴': 67, '是不是': 68, '醫院': 69, '有點': 70, '當時': 71, '壓力': 72, '小時': 73, '痛': 74, '。」': 75, '下': 76, '師': 77, '下來': 78, '不可': 79, '過去': 80, '嚴重': 81, '關係': 82, '家': 83, '離開': 84, '想到': 85, '完全': 86, '焦慮': 87, '面對': 88, '話': 89, '發生': 90, '好累': 91, '謝謝': 92, '聽': 93, '突然': 94, '「': 95, '一下': 96, '沒辦': 97, '。': 98, '所有': 99, '病': 100, '繼續'

### Load the LSTM model

In [11]:
model = tf.keras.models.load_model('suicide_model_2_classes.h5')

#Test the model

### predict function

In [12]:
def predict_risk(input_text):
  test_data_df = pd.DataFrame({'ID': [0], 'Description':[input_text], 'Class':[2]}) #'Class' is a temp number, dont worry about it
  # 斷詞處理
  test_df = JeibaCutWords(test_data_df, "Description")
  #extract sentence
  test_corpus = process_df(test_df)
  # print(f"total cases: {len(test_corpus)}")
  print("test_corpus:", test_corpus)

  sentences_test=[]
  for x in range(len(test_corpus)):
      sentences_test.append(test_corpus[x][0])

  sequences_test = tokenizer.texts_to_sequences(sentences_test)
  padded_test = pad_sequences(sequences_test, maxlen=max_length, padding=padding_type, truncating=trunc_type)

  result = model.predict_classes(padded_test)
  print("predict_classes:", result)
  if result[0][0]==1:
    return "辨識結果: 有風險"
  else:
    return "辨識結果: 低風險"

### predict, put test text in "input_text"

In [14]:
input_text = "習慣了。現在漸漸地變成了一個常態只是後續要處理好自己還是人類部分的情感需求要漸漸地漸漸地習慣讓這部分死去直到我完全的成為了活屍我就成功了對她人最安全的做法就是我完全的不在對方的世界裏我也完全的不說話認識一個人。馬上把她刪好友是對對方最好的做法也是對我最好的做法沒有接觸。沒有期待 沒有傷害快樂是別人的 我什麼都沒有如果＊我＊的這概念也消失了那我就連這段話都感覺不到了這是我的目標成為一具屍體。也不知道賺錢做什麼我覺得世界是一座巨大的墳墓一切都在腐爛我是具有溫度的屍體感覺可以死了。可是又好像跟死了差不多說出這句話竟有種矛盾的感覺反正也離死不遠了--"
# input_text = "今天天氣真好，想去走走"
predict_risk(input_text)

Building prefix dict from /content/suicide_multiclass/Jeiba/dict.txt.big ...
Loading model from cache /tmp/jieba.ucb66ba17c4739edbeb5fb122bb3a9bb9.cache
Loading model cost 1.264 seconds.
Prefix dict has been built successfully.


test_corpus: [['習慣 漸漸 漸地 變成 常態 後續 處理 理好 人類 部分 情感 需求 漸漸 漸地 漸漸 漸地 習慣 這部 這部分 部分 死去 直到 完全 成 屍 成功 最 安全 做法 完全 不在 對方 裏 完全 說話 認識 刪 好友 對對 對方 最好 做法 最好 做法 接觸 有期 期待 有傷 傷害 快樂 有如 概念 消失 這段 這段話 不到 目標 成 一具 屍體 賺錢 一座 巨大 墳墓 一切都在 腐爛 具有 溫度 屍體 死 死 不多 說出 句 話 有種 矛盾 反正 死 不遠', 2]]
predict_classes: [[1]]


'辨識結果: 有風險'